In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import nibabel as nib

from tensorflow import keras
from tensorflow.keras import layers

from scipy import ndimage

In [2]:
scan_paths = [
    os.path.join(os.getcwd(), "../BET_BSE_DATA", x)
    for x in os.listdir("../BET_BSE_DATA")
]

print("Brain scans: " + str(len(scan_paths)))

Brain scans: 4


In [3]:
def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan


def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume

In [13]:
import csv
 
# csv file name
filename = "/home/joelkik/DataMining/BET_BSE_DATA/Label_file.csv"
 
# initializing the titles and rows list
fields = []
labels_facical_feature_dict = dict()
labels_brain_feature_dict = dict()
 
# reading csv file
with open(filename, 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        #print(row)
        if row[1].lower() == 'yes':
            labels_facical_feature_dict[row[0]+'.gz'] = 1
        else:
            labels_facical_feature_dict[row[0]+'.gz'] = 0
        if row[2].lower() == 'yes':
            labels_brain_feature_dict[row[0]+'.gz'] = 1
        else:
            labels_brain_feature_dict[row[0]+'.gz'] = 0
 
    # get total number of rows
    print("Total no. of rows: %d"%(csvreader.line_num))
 
# printing the field names
print('Field names are:' + ', '.join(field for field in fields))
#print(labels_dict)

Total no. of rows: 1312
Field names are:Filename, Recognizable-Facial-Feature, Brain-Feature-Loss


In [14]:
brain_scans=[]
labels_facial_feature=[]
labels_brain_feature=[]
labels_to_file=[]
for path in os.listdir("/home/joelkik/DataMining/BET_BSE_DATA/files")[:20]:
    brain_scans.append(process_scan("/home/joelkik/DataMining/BET_BSE_DATA/files/"+path))
    labels_facial_feature.append(labels_facical_feature_dict[path])
    labels_brain_feature.append(labels_brain_feature_dict[path])
    labels_to_file.append(path)

In [15]:
brain_scans_np = np.array(brain_scans)
labels_facial_feature_np = np.array(labels_facial_feature)

In [16]:
model = keras.models.load_model("/home/joelkik/DataMining/CPSC8650_Project/3d_brain_image_classification.h5")

In [17]:
for i in range(len(brain_scans)):
    prediction = model.predict(np.expand_dims(brain_scans_np[i], axis=0))[0]
    print("Scan File: ", labels_to_file[i], ", Actual: ", labels_facial_feature_np[i], ", Predicted: ",round(float(prediction)))

Scan File:  IXI476-IOP-1140-T1_bse_less_s47_r1.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI048-HH-1326-T1_bse_less_s44_r1.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI554-Guys-1068-T1_bet_18.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI115-Guys-0738-T1_bse_high_s76_r2.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI156-Guys-0837-T1_bse_less_s42_r1.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI218-HH-1815-T1_bse_high_s74_r2.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI038-Guys-0729-T1_bet_85.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI464-IOP-1029-T1_bse_default.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI435-IOP-1040-T1_bet_82.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI350-Guys-0908-T1_bet_82.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI013-HH-1212-T1_bet_19.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI607-Guys-1097-T1_bse_less_s56_r1.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI350-Guys-0908-T1_bet_1.nii.gz , Act

In [18]:
model = keras.models.load_model("/home/joelkik/DataMining/CPSC8650_Project/3d_brain_feature_loss_model.h5")

In [19]:
for i in range(len(brain_scans)):
    prediction = model.predict(np.expand_dims(brain_scans_np[i], axis=0))[0]
    print("Scan File: ", labels_to_file[i], ", Actual: ", labels_brain_feature[i], ", Predicted: ",round(float(prediction)))

Scan File:  IXI476-IOP-1140-T1_bse_less_s47_r1.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI048-HH-1326-T1_bse_less_s44_r1.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI554-Guys-1068-T1_bet_18.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI115-Guys-0738-T1_bse_high_s76_r2.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI156-Guys-0837-T1_bse_less_s42_r1.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI218-HH-1815-T1_bse_high_s74_r2.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI038-Guys-0729-T1_bet_85.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI464-IOP-1029-T1_bse_default.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI435-IOP-1040-T1_bet_82.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI350-Guys-0908-T1_bet_82.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI013-HH-1212-T1_bet_19.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI607-Guys-1097-T1_bse_less_s56_r1.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI350-Guys-0908-T1_bet_1.nii.gz , Act